# Information Retrieval I #
## Assignment 2: retrieval models [100 points + 10 bonus points] ##
**TA**: Christophe Van Gysel (cvangysel@uva.nl; C3.258B, Science Park 904)

**Secondary TAs**: Harrie Oosterhuis, Nikos Voskarides

In this assignment you will get familiar with basic information retrieval concepts. You will implement and evaluate different information retrieval ranking models and evaluate their performance.

We provide you with a VirtualBox image that comes pre-loaded with an index and a Python installation. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-two assignment**, the deadline is **23:59 - 25 January, 2017**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](https://piazza.com/class/ixoz63p156g1ts).

### Technicalities (must-read!) ###
This assignment comes pre-loaded on a VirtualBox running Ubuntu. We have configured the indexing software and Python environment such that it works out of the box. You are allowed to extract the files from the VirtualBox and set-up your own non-virtualized environment. However, in this case you are on your own w.r.t. software support.

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.
      
`Python + Jupyter`, `Indri`, `Gensim` and `Pyndri` come pre-installed (see `$HOME/.local`). TREC Eval can be found in `$HOME/Downloads/trec_eval.9.0`. The password of the `student` account on the VirtualBox is `datascience`.

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [1]:
import logging
import sys

def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [19]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [20]:
# print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))
# print(index.maximum_document())
# print(index.document_base())
# print(index.maximum_document()-index.document_base())


Let's take the first document out of the collection and take a look at it:

In [21]:
# example_document = index.document(index.document_base())
# # print(example_document)

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [22]:
token2id, id2token, _ = index.get_dictionary()
# print(list(id2token.items())[:15])

Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [23]:
# print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [24]:
# query_tokens = index.tokenize("University of Massachusetts")
# print("Query by tokens:", query_tokens)
# query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
# print("Query by ids with stopwords:", query_id_tokens)
# query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
# print("Query by ids without stopwords:", query_id_tokens)
# print(token2id.get('university'))

Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [25]:
# matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
# print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
# print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [26]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

### Task 1: Implement and compare lexical IR methods [45 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html). **[5 points]**
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.2, ..., 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, ..., 2000]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.2, ..., 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[10 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

## Setups

In [27]:
import time
import pickle
import math
from itertools import islice
import random

In [28]:
n = index.maximum_document()-index.document_base()
def get_rid_of_zeros(n):
    collection = []
    for i in range(1,n+1):
        collection.append([word for word in index.document(i)[1] if word > 0])
    return collection

In [29]:
# collection = get_rid_of_zeros(n)
# pickle.dump(collection, open("./results/collection.p", "wb"))

In [30]:
collection = pickle.load(open( "./results/collection.p", "rb"))

In [31]:
def get_collection_length():
    length_collection = 0
    for i in range(n):
        length_collection += len(collection[i])
    return length_collection

In [32]:
col_len = get_collection_length()

In [33]:
def get_unique_collection(n):
    unique_words_docs = []
    for doc in collection:
        unique_words_docs.append(list(set(doc)))
    return unique_words_docs

In [34]:
# unique_words_docs = get_unique_collection(n)
# pickle.dump(unique_words_docs, open("./results/unique_words_docs.p", "wb"))

In [35]:
unique_words_docs = pickle.load(open( "./results/unique_words_docs.p", "rb"))

In [36]:
# the query ids of the validation list
with open('./ap_88_89/qrel_validation', 'r') as val_queries: 
    val_queries = list(set([line.split(' ')[0] for line in val_queries]))

** Inverted Index List **

In [37]:
queries_dict = {} # {qid: qstring, qid: qstring...}
with open('./ap_88_89/topics_title', 'r') as f_topics: 
    for query in parse_topics([f_topics]).items():
        queries_dict[query[0]] = query


In [38]:
def get_inverted_index():
    inverted_list = {}
    query_list = []

    with open('./ap_88_89/topics_title', 'r') as f_topics:   
        for query in parse_topics([f_topics]).items():
            query_list.append(query)

    nr = 0
    for query in query_list:
        nr += 1
        if nr % 10 == 0: #
            print(str(nr)+'/'+str(len(query_list)))

        # getting the query term ids
        query_id = query[0]
        query_string = query[1]
        query_tokens = index.tokenize(query_string)
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

        for qti in query_id_tokens: # for every query term
            if qti not in inverted_list: # only check unique query token once
                tot_count = 0
                inverted_list[qti] = [[],0]
                for i in range(n):                
                    word_counter = collection[i].count(qti)
                    if word_counter > 0:
                        tot_count += word_counter
                        docid = index.document_ids([index.document(i+1)[0]])[0][1]
                        inverted_list[qti][0].append(docid) # add document to query tok id
                inverted_list[qti][1]= (tot_count)
    return inverted_list, query_list

Get lists:

In [39]:
# inverted_list, query_list = get_inverted_index()
# pickle.dump(inverted_list, open("./results/inverted_index.p", "wb"))
# pickle.dump(query_list, open("./results/query_list.p", "wb"))

Load lists:

In [40]:
inverted_list = pickle.load(open( "./results/inverted_index.p", "rb"))
# query_list = pickle.load(open( "./results/query_list.p", "rb"))

In [52]:
def make_dict_format(dic):
    results = []
    for doc, score in dic.items():
        results.append((score, doc))
    return tuple(results)

In [27]:
def get_query_val_docs():
    query_val_docs = {}
    nr = 0
    for query_id in val_queries:
        nr +=1
        if nr % 5 ==0:
            print('doc', nr)
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

        relev_documents = []
        for token in query_id_tokens:
            for doc in inverted_list[token][0]:
                if doc not in relev_documents:
                    relev_documents.append(doc)

        query_val_docs[query_id] = relev_documents
    return query_val_docs

In [28]:
# query_val_docs = get_query_val_docs()
# pickle.dump(query_val_docs, open("./results/query_val_docs.p", "wb"))

In [17]:
query_val_docs = pickle.load(open( "./results/query_val_docs.p", "rb"))

# Retrieval Models

## TF-IDF  (Vector-space)

In [41]:
def idf(t):
    return math.log(n)-math.log(len(inverted_list[t][0]))

In [42]:
def tf_idf(t, d):
    return math.log(1+collection[d].count(t)) * idf(t)

In [43]:
def score_TFIDF(q,d):
    unique = unique_words_docs[d-1]
    score = 0
    for word in q:
        score += tf_idf(word,d-1)   
    return score

In [32]:
def get_TFIDF_scores():
    TFIDF_dict = {}
    
    nr = 0
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = score_TFIDF(query_id_tokens, d)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            i = str(index.document(i)[0])
            r[i] = 0 if i not in r else r[i]

        TFIDF_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return TFIDF_dict

Get scores:

In [33]:
# TFIDF_scores = get_TFIDF_scores()
# pickle.dump(TFIDF_scores, open("./results/tfidf_results.p", "wb"))

Load scores:

In [34]:
TFIDF_scores = pickle.load(open( "./results/tfidf_results.p", "rb"))

## BM25 (Probabilistic)

In [35]:
def average_length():
    l = 0
    for i in range(n):
        l += len(collection[i])
    return l/float(n)

l_av = average_length()

In [36]:
def BM25(t,d):
    k1 = 1.2
#     k3 = 1.2
    b = 0.75
    first = ((k1+1)* collection[d].count(t)) / (k1*((1-b)+b*(len(collection[d])/l_av))+collection[d].count(t))
#     second = ((k3+1)*tf(query_id_tokens,t))/(k3*tf(query_id_tokens, t))
    return  first*idf(t)

In [37]:
def score_BM25(q,d):
    unique = unique_words_docs[d-1]
    score = 0
    for word in set(q):
        score += BM25(word,d-1)
        
    return score

In [38]:
def get_BM25_scores():
    BM25_dict = {}
    
    nr = 0
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = score_BM25(query_id_tokens, d)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            i = str(index.document(i)[0])
            r[i] = 0 if i not in r else r[i]

        BM25_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return BM25_dict

Get scores:

In [39]:
BM25_scores = get_BM25_scores()
pickle.dump(BM25_scores, open("./results/bm25_results.p", "wb"))

Load scores:

In [40]:
BM25_scores = pickle.load(open( "./results/bm25_results.p", "rb"))

**Background probability**


In [50]:
def background_prob(w):
    if w in inverted_list:
        tf_w_C = inverted_list[w][-1]
    else: 
        tf_w_C = 0
    return tf_w_C/float(col_len)

## Smoothing

## Jelinek-Mercer


In [67]:
def jelinek_mercer(lamb, d, w):
    if len(collection[d]) == 0:
        print(d)
    P = (lamb * (collection[d].count(w)/len(collection[d]))) + ((1-lamb) * background_prob(w))
    return P

In [68]:
def jelinek_score(lamb,query,d):
    score = 0
    for q in query:
        score += math.log(jelinek_mercer(lamb,d,q))
    return score

In [69]:
def get_jelinek_scores(lamb):
    print('\r','lamb:',lamb)
    jelinek_dict = {}
    nr = 0
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
        
    
        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = jelinek_score(lamb, query_id_tokens, d-1)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            j = str(index.document(i)[0])
            r[j] = jelinek_score(lamb, query_id_tokens, i-1) if j not in r else r[j]

        jelinek_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return jelinek_dict

Getting scores:

In [70]:
# jelinek_scores_0_1 = get_jelinek_scores(0.1)
# jelinek_scores_0_2 = get_jelinek_scores(0.2)
# jelinek_scores_0_3 = get_jelinek_scores(0.3)
# jelinek_scores_0_4 = get_jelinek_scores(0.4)
# jelinek_scores_0_5 = get_jelinek_scores(0.5)
# jelinek_scores_0_6 = get_jelinek_scores(0.6)
# jelinek_scores_0_7 = get_jelinek_scores(0.7)
# jelinek_scores_0_8 = get_jelinek_scores(0.8)
# jelinek_scores_0_9 = get_jelinek_scores(0.9)

 lamb: 0.6
 lamb: 0.7
 lamb: 0.8
 lamb: 0.9
 29/30 

In [71]:
# pickle.dump(jelinek_scores_0_1, open("./results/jelinek_scores_0_1.p", "wb"))
# pickle.dump(jelinek_scores_0_2, open("./results/jelinek_scores_0_2.p", "wb"))
# pickle.dump(jelinek_scores_0_3, open("./results/jelinek_scores_0_3.p", "wb"))
# pickle.dump(jelinek_scores_0_4, open("./results/jelinek_scores_0_4.p", "wb"))
# pickle.dump(jelinek_scores_0_5, open("./results/jelinek_scores_0_5.p", "wb"))
# pickle.dump(jelinek_scores_0_6, open("./results/jelinek_scores_0_6.p", "wb"))
# pickle.dump(jelinek_scores_0_7, open("./results/jelinek_scores_0_7.p", "wb"))
# pickle.dump(jelinek_scores_0_8, open("./results/jelinek_scores_0_8.p", "wb"))
# pickle.dump(jelinek_scores_0_9, open("./results/jelinek_scores_0_9.p", "wb"))

Loading scores:

In [ ]:
jelinek_scores_0_1 = pickle.load(open( "./results/jelinek_scores_0_1.p", "rb"))
jelinek_scores_0_2 = pickle.load(open( "./results/jelinek_scores_0_2.p", "rb"))
jelinek_scores_0_3 = pickle.load(open( "./results/jelinek_scores_0_3.p", "rb"))
jelinek_scores_0_4 = pickle.load(open( "./results/jelinek_scores_0_4.p", "rb"))
jelinek_scores_0_5 = pickle.load(open( "./results/jelinek_scores_0_5.p", "rb"))
jelinek_scores_0_6 = pickle.load(open( "./results/jelinek_scores_0_6.p", "rb"))
jelinek_scores_0_7 = pickle.load(open( "./results/jelinek_scores_0_7.p", "rb"))
jelinek_scores_0_8 = pickle.load(open( "./results/jelinek_scores_0_8.p", "rb"))
jelinek_scores_0_9 = pickle.load(open( "./results/jelinek_scores_0_9.p", "rb"))

## Dirichlet Prior

In [78]:
def dirichlet_prior(mu,d,w):
    return ((len(collection[d])/(len(collection[d])+mu)) * (collection[d].count(w)/len(collection[d]))) + ((mu/(mu+len(collection[d]))) * background_prob(w))

In [79]:
def dirichlet_score(mu,query,d):
    score = 0
    for q in query:
        score += math.log(dirichlet_prior(mu,d,q))
    return score

In [80]:
def get_dirichlet_scores(mu):
    print('\r','mu:',mu)
    dirichlet_dict = {}
    nr = 0
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]      
    
        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = dirichlet_score(mu, query_id_tokens, d-1)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            j = str(index.document(i)[0])
            r[j] = dirichlet_score(mu, query_id_tokens, i-1) if j not in r else r[j]

        dirichlet_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return dirichlet_dict

Getting scores:

In [85]:
# dirichlet_scores_500 = get_dirichlet_scores(500)
# dirichlet_scores_1000 = get_dirichlet_scores(1000)
# dirichlet_scores_1500 = get_dirichlet_scores(1500)
# dirichlet_scores_2000 = get_dirichlet_scores(2000)

In [84]:
# pickle.dump(dirichlet_scores_500, open("./results/dirichlet_scores_500.p", "wb"))
# pickle.dump(dirichlet_scores_1000, open("./results/dirichlet_scores_1000.p", "wb"))
# pickle.dump(dirichlet_scores_1500, open("./results/dirichlet_scores_1500.p", "wb"))
# pickle.dump(dirichlet_scores_2000, open("./results/dirichlet_scores_2000.p", "wb"))

Loading scores:

In [ ]:
dirichlet_scores_500 = pickle.load(open( "./results/dirichlet_scores_500.p", "rb"))
dirichlet_scores_1000 = pickle.load(open( "./results/dirichlet_scores_1000.p", "rb"))
dirichlet_scores_1500 = pickle.load(open( "./results/dirichlet_scores_1500.p", "rb"))
dirichlet_scores_2000 = pickle.load(open( "./results/dirichlet_scores_2000.p", "rb"))


**Absolute Discounting**

In [92]:
def absolute_discounting(delta, d, w):
    if len(collection[d]) == 0:
        print(d)
    return (max(collection[d].count(w)-delta, 0)/len(collection[d])) + (((delta * len(unique_words_docs[d]))/len(collection[d])) * (background_prob(w)))

In [93]:
def AD_score(delta,query,d):
    score = 0
    for q in query:
        score += math.log(absolute_discounting(delta,d,q))
    return score

In [94]:
def get_AD_scores(delta):
    print('\r','delta:',delta)
    AD_dict = {}
    nr = 1
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]      
    
        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = AD_score(delta, query_id_tokens, d-1)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            j = str(index.document(i)[0])
            r[j] = AD_score(delta, query_id_tokens, i-1) if j not in r else r[j]

        AD_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return AD_dict

In [ ]:
AD_scores_0_1 = get_AD_scores(0.1)
AD_scores_0_2 = get_AD_scores(0.2)
AD_scores_0_3 = get_AD_scores(0.3)
AD_scores_0_4 = get_AD_scores(0.4)
AD_scores_0_5 = get_AD_scores(0.5)
AD_scores_0_6 = get_AD_scores(0.6)
AD_scores_0_7 = get_AD_scores(0.7)
AD_scores_0_8 = get_AD_scores(0.8)
AD_scores_0_9 = get_AD_scores(0.9)

 delta: 0.1
 23/30 

## Positional Language Models

In [ ]:
query_terms =[]
for query in query_list:
    query_tokens = index.tokenize(query[1])
    query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
    query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

    for query_token in query_id_tokens:
        if query_token not in query_terms:
            query_terms.append(query_token)

In [ ]:
# q_docs = []
# nr = 0
# for doc in collection:
#     nr +=1
#     if nr % 1000 ==0:
#         print(nr)
#     query_doc = []
#     for i in range(len(doc)):
#         if doc[i] in query_terms:
#             for qterm in query_terms:
#                 if qterm == doc[i]:
#                     query_doc.append([qterm, i])
#     q_docs.append(query_doc)               

In [ ]:
# pickle.dump(q_docs, open("./results/query_docs.p", "wb"))

In [ ]:
def kernel_gaussian(sigma,i,j):
    return math.exp((-1*((i-j)**2))/(2*(sigma**2)))
    
def kernel_triangle(sigma,i,j):
    if i-j <= sigma:
        return 1-((i-j)/sigma)
    else:
        return 0.0

def kernel_cosine(sigma,i,j):
    if i-j <= sigma:
        return 0.5*(1+math.cos(((i-j)*math.pi)/sigma))
    else:
        return 0.0

def kernel_circle(sigma,i,j):
    if i-j <= sigma:
        return math.sqrt(1-(((i-j)/sigma)**2))
    else:
        return 0.0

def kernel_passage(sigma,i,j):
    if i-j <= sigma:
        return 1.0
    else:
        return 0.0  

In [ ]:
def c(w,j,d): 
    if w == d[j]:
        return 1
    else: return 0

In [ ]:
def c_prime(w,i,d): # 0.0003 seconds
    c_prime = 0
    
    for query in q_docs[d]:
        if query[0] == w:
            j = query[1]
            c_prime += kernel_gaussian(50,i,j)
        
#     for j in range(len(collection[d])): 
#         c_prime += c(w,j,collection[d])*kernel_gaussian(50,i,j)
    return c_prime

In [ ]:
def Z(i,d): # 0.25 seconds
    Z = 0
    for word in unique_words_docs[d]:
        Z += c_prime(word,i,d)
    return Z
    

In [ ]:
def get_all_zs():
    Z = []
    
    max_len = 0
    for doc in collection:
        if len(doc) > max_len:
            max_len = len(doc)
    
    for i in range(max_len):
        z = 0
        for j in range(max_len):
            z += kernel_gaussian(50,i,j)
        Z.append(z)
    return Z


In [ ]:
# Zs = get_all_zs()

In [ ]:
def PLM(mu,w,i,d): # 0.25 seconds
    P = (c_prime(w,i,d) + (mu * background_prob(w))) / (Zs[i] + mu)#(Z(i,d) + mu)
    return P

In [9]:
def PLM_score(q,d):
#     unique = list(set(collection[d-1]))
    unique = unique_words_docs[d-1]
    max_score = -100000
    for i in range(len(collection[d-1])):   
        scores = []
        for word in q:
            if inverted_list[word][0]:
                scores.append(((q.count(word)/float(len(q))) * (math.log((q.count(word)/float(len(q)))/PLM(1000,word,i,d-1)))))
        if -sum(scores) > max_score:
            max_score = -sum(scores)
    return max_score

In [ ]:
def get_PLM_scores():
#     nr = 0
    
    PLM_dict = {}
    secdoc = 0
    for query in query_list:
        query_id = query[0]
        query_tokens = index.tokenize(query[1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
        relev_documents = []
        for token in query_id_tokens:
            for doc in inverted_list[token][0]:
                if doc not in relev_documents:
                    relev_documents.append(doc)
        print('Q'+str(query_id), 'estimated time: '+ str(round((secdoc*len(relev_documents))/60.0,2)), 'min')
        if relev_documents:
            PLM_dict[str(query_id)] = []
            start = time.time()
            for d in relev_documents:
                PLM_dict[str(query_id)].append((PLM_score(query_id_tokens, d), str(index.document(d)[0])))
            PLM_dict[str(query_id)] = tuple(PLM_dict[str(query_id)])
            secdoc = (time.time()-start)/float(len(relev_documents))
        output = open("./results/PLM_parts.p", "wb")
        pickle.dump(PLM_dict, output)
        output.close()
    return PLM_dict

    PLM_dict = {}
    secdoc = 0
    
    for query_id in val_queries:
        r = {}
        print('\r',str(nr)+'/'+str(len(val_queries)), end=" ")
        query_tokens = index.tokenize(queries_dict[str(query_id)][1])
        query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
        query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]

        for d in query_val_docs[query_id]:
            d_id = str(index.document(d)[0])
            r[d_id] = score_BM25(query_id_tokens, d)
            
        while len(r) < 1000:
            i = random.randrange(1,n)
            i = str(index.document(i)[0])
            r[i] = 0 if i not in r else r[i]

        BM25_dict[str(query_id)] = make_dict_format(r)
        nr += 1
        
    return BM25_dict



In [ ]:
PLM_scores = get_PLM_scores()
pickle.dump(PLM_scores, open("./results/plm_results.p", "wb"))


In [ ]:
# new_dict = pickle.load(open( "./results/plm_results.p", "rb"))
# print(new_dict)

### Task 2: Latent Semantic Models (LSMs) [25 points + 10 bonus points] ###

In this task you will experiment with applying a distributional semantics methods ([word2vec](http://arxiv.org/abs/1411.2738)  **[5 points]**, [LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]**, [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]** and [doc2vec](https://cs.stanford.edu/~quocle/paragraph_vector.pdf) **[5 points]**) for retrieval.

You do not need to implement word2vec, LSI, LDA and doc2vec on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html) (pre-loaded on the VirtualBox). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. For example, in the case of word2vec, you only have vectors for individual words and not for documents or phrases. Try one of the following methods for producing these representations:
   * Average or sum the word vectors.
   * Cluster words in the document using [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and use the centroid of the most important cluster. Experiment with different values of K for k-means.
   * Using the [bag-of-word-embeddings representation](https://ciir-publications.cs.umass.edu/pub/web/getpdf.php?id=1248). **[10 bonus points]**
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [ ]:
# # Provide right gensim input with pyndri for word2vec, this code is provided with the assignment.
# # IndriSentences gives sentences in the right format for word2vec.
# # It works as a bridge between gensim and pyndri -> we have to do this for the other distributional semantic methods as well.
# # Understanding which input parameters in what format that are needed for each model shows understanding of the model itself.

# import pyndri.compat
# import gensim
# import copy
# import logging
# word2vec_init = gensim.models.Word2Vec(
#     size=64,  # Embedding size - initially 300
#     window=5,  # One-sided window size, init was 5
#     sg=True,  # Skip-gram.
#     min_count=1,  # Minimum word frequency.
#     sample=1e-3,  # Sub-sample threshold.
#     hs=False,  # Hierarchical softmax.
#     negative=10,  # Number of negative examples.
#     iter=1,  # Number of iterations.
#     workers=8,  # Number of workers.
# )

# dictionary = pyndri.extract_dictionary(index)
# sentences = pyndri.compat.IndriSentences(index, dictionary, max_documents=200) # max_documents due to time complexity

# logging.info('Constructing word2vec vocabulary.')

# # Build vocab.
# word2vec_init.build_vocab(sentences, trim_rule=None)

# models = [word2vec_init]

# for epoch in range(1, 6):#range(1, 5 + 1):
#     logging.info('Epoch %d', epoch)
#     model = copy.deepcopy(models[-1])
#     model.train(sentences)

#     models.append(model)

# logging.info('Trained models: %s', models)
# print(models[-1]) # last model is most advanced

# ### END OF CODE AS PROVIDED BY ASSIGNMENT


In [ ]:

# list(token2id.keys())[:20]
# models[-1].most_similar(positive=['king'], negative=['man'])
# ## LSI ##
# '''LSI compares how often words appear together in the same document
# and compares how often those occurences happen in all of the documents 
# that the dictionary has in its index.'''

# # Get docs as bag of words #
# # print(id2token[0]) is not possible, zeroeth token = irrelevant token. Have to leave this out. xxx
# def getdocs(index, max_docs):
#     dictionary = pyndri.extract_dictionary(index)
#     bowdoc=[]
#     if max_docs == None:
#         for document in range(index.document_base(), index.maximum_document()):  
#             bowdoc.append(dictionary.doc2bow(index.document(document)[1])[1:]) #[1:] throws out irrelevant word '0'
#     else:
#         for document in range(index.document_base(), index.document_base()+max_docs):  
#             bowdoc.append(dictionary.doc2bow(index.document(document)[1])[1:])
#     return bowdoc
# bowdoc = getdocs(index, 2)
# #print(bowdoc)
# print("got bow")
# lsimodel = gensim.models.LsiModel(bowdoc, 2, id2token, 2, 1.0, False, True, 2, 100)
# print("trained model")


# print(lsimodel.show_topic(1, topn=1))




### Task 3: Learning to rank (LTR) [10 points] ###

In this task you will get an introduction into learning to rank for information retrieval. You will experiment with a pointwise learning to rank method, logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

**NOTE**: you can only perform this task if you have completely finished Task 1 and Task 2.

In this experiment, you will use the retrieval methods you implemented in Task 1 and Task 2 as features for the learning to rank model. Train your LTR model using 10-fold cross validation on the test set. For every query, first create a document candidate set using the top-1000 documents using TF-IDF. Secondly, compute query-document values using the retrieval models above and use them as features. Note that the feature values of different retrieval methods are likely to be distributed differently.

Your approach will definitely not be as good as the state-of-the-art since you are taking a pointwise approach, but we do not ask you to try pair- or listwise methods because they will be the main topic of the next assignment.

In [ ]:
# from sklearn.linear_model import LogisticRegression

### Task 4: Write a report [20 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Do not send us the VirtualBox, but only the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file system structure as on the VirtualBox.
